## This code finds crossover of a particular rate across a period of time

In [ ]:
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pytz


In [ ]:

try:
    mt5.shutdown()
except Exception as e:
    print(e)

# --- Parameters ---
symbol = "XAUUSDm"
timeframe = mt5.TIMEFRAME_M30  # Options: TIMEFRAME_M1, M5, H1, D1, etc.
threshold = 3325.535
bars = 500  # number of historical bars

# --- Initialize MT5 ---
if not mt5.initialize():
    raise RuntimeError("initialize() failed")

# Ensure symbol is available
if not mt5.symbol_select(symbol, True):
    raise RuntimeError(f"Failed to select symbol {symbol}")

# --- Get historical data ---
rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, bars)

timezone = pytz.timezone("Etc/UTC")
date_to = datetime.now(timezone)
# print(date_to)
date_from = date_to - timedelta(days=50)
range_rates = mt5.copy_rates_range(symbol, timeframe, date_from, date_to)
df_range_rates = pd.DataFrame(range_rates)
# print(df_range_rates.shape)

if rates is None:
    raise RuntimeError("No data returned")

# --- Convert to DataFrame ---
df = pd.DataFrame(range_rates) #rates)
df['time'] = pd.to_datetime(df['time'], unit='s')

# --- Filter by Close price ---
filtered = df[df['close'] > threshold]
print(filtered.shape)
# filtered['Profit'] = filtered['close'] - threshold * 0.1
# filtered.loc[filtered['close'] > 0.0 ,'Profit'] = (filtered['close'] - threshold) * (0.1*10.0)
print(filtered.shape)
# print(filtered)

# --- Display filtered data ---
print(filtered[['time', 'close']])

# --- Plot ---
plt.figure(figsize=(12, 6))
# Plot price line
plt.plot(df['time'], df['close'], label='Close Price')
# Plot red dots
plt.scatter(filtered['time'], filtered['close'], color='red', label=f'> {threshold}', zorder=5)

# --- Annotate each red dot ---
for idx, row in filtered.iterrows():
    time = row['time']
    price = row['close']
    profit = (price - threshold) * 0.1 
    profit = profit *10.0
    label_y = min(price-2,price + y_offset)
    
    # Add annotation with slight offset
    plt.annotate(f"{profit :.2f}", (time, label_y),
                 textcoords="offset points", xytext=(0,10), ha='center',
                 fontsize=8, color='darkred', rotation=45)
    # plt.annotate(price, (time, price),
    #              textcoords="offset points", xytext=(0,10), ha='center',
    #              fontsize=8, color='darkred', rotation=45)

''' Add vertical/horizontal lines for reference (optional) '''
# plt.axvline(x=order_time, color='gray', linestyle='--', linewidth=1)
# plt.axhline(y=threshold, color='gray', linestyle='--', linewidth=1)
# plt.xlim(filtered['time'].min(), filtered['time'].max())
# plt.ylim(min(filtered['close'].min(), threshold) * 0.99, max(filtered['close'].max(), threshold) * 1.01)

# Add marker
order_time = datetime(2025, 7, 29, 9, 56)
plt.scatter(order_time, threshold, color='green', marker='X', s=100, label='Order Point', zorder=5)
plt.annotate(f"{threshold:.4f}",
             (order_time, threshold),
             textcoords="offset points",
             xytext=(0, 10),
             ha='center',
             fontsize=9,
             color='green')

# Final layout
plt.xlabel("Time")
plt.ylabel("Price")
plt.title(f"{symbol} Close Prices (>={threshold})")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Shutdown MT5 connection ---
mt5.shutdown()


In [ ]:
filtered['dateseries'] = filtered['time'].dt.date
filtered

In [ ]:
filtered.shape

In [ ]:
# Keep the first row per unique date
dedup = filtered.drop_duplicates(subset='dateseries',keep='first')

In [ ]:
print(dedup.shape)

In [ ]:
dedup